<a href="https://colab.research.google.com/github/HSYhrae/TIL/blob/master/Random_Forest(car).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 랜덤포레스트
- 결정 트리의 단점인 오버피팅 문제를 완화시켜주는 발전된 형태의 트리 모델이다.
- 앙상블 기법을 사용한 트리 기반 모델 중 가장 보편적인 방법이다.
- 부스팅 모델에 비해 예측력이나 속도가 부족하지만, 부스팅 모델을 이해하기 위해선 필수적인 알고리즘이다.

### 문제정의
- 중고차 판매 이력 데이터셋을 이용해 중고차 가격을 예측

### 라이브러리 및 데이터 불러오기

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

url = 'https://media.githubusercontent.com/media/musthave-ML10/data_source/main/car.csv'
data = pd.read_csv(url)

### 데이터 확인하기

In [22]:
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


#### 변수 설명
- 독립변수
    1. name: 이름
    2. year: 생산년
    3. km_driven: 주행거리
    4. fuel: 연료
    5. seller_type: 판매자유형
    6. transmission: 변속기
    7. owner: 차주 변경 내역
    8. mileage: 마일리지
    9. engine: 배기량
    10. max_power: 최대출력
    11. torque: 토크
    12. seats: 인승
- 종속변수
    + selling_price: 판매가

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine         7907 non-null   object 
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 825.6+ KB


- 통계정보 확인
- 판매가에서 max, min 모두 이상치로 보여짐
- 하지만 트리 모델을 사용하므로 별도 처리X

In [24]:
round(data.describe(), 2)

,year,selling_price,km_driven,seats
count,8128.00,8128.00,8128.00,7907.00
mean,2013.80,638271.81,69819.51,5.42
std,4.04,806253.40,56550.55,0.96
min,1983.00,29999.00,1.00,2.00
25%,2011.00,254999.00,35000.00,5.00
50%,2015.00,450000.00,60000.00,5.00
75%,2017.00,675000.00,98000.00,5.00
max,2020.00,10000000.00,2360457.00,14.00


### 데이터 전처리: 텍스트 데이터

- engine 변수 전처리

In [25]:
data[['engine']]

,engine
0,1248 CC
1,1498 CC
2,1497 CC
3,1396 CC
4,1298 CC
...,...
8123,1197 CC
8124,1493 CC
8125,1248 CC
8126,1396 CC


- 공백을 기준으로 앞의 숫자와 문자를 별도의 변수로  분리

In [26]:
data['engine'].str.split(expand = True)

,0,1
0,1248,CC
1,1498,CC
2,1497,CC
3,1396,CC
4,1298,CC
...,...,...
8123,1197,CC
8124,1493,CC
8125,1248,CC
8126,1396,CC


- 숫자형 변수는 engine에 할당
- 문자형 변수는 engine_unit에 할당

In [27]:
data[['engine', 'engine_unit']] = data['engine'].str.split(expand = True)

- engine 변수 확인

In [28]:
data['engine'].head()

0    1248
1    1498
2    1497
3    1396
4    1298
Name: engine, dtype: object

- 데이터 타입을 숫자형으로 변환

In [29]:
data['engine'] = data['engine'].astype('float32')

- 변환됐는지 확인

In [30]:
data['engine'].head()

0    1248.0
1    1498.0
2    1497.0
3    1396.0
4    1298.0
Name: engine, dtype: float32

- engine_unit 변수 분포 확인

In [31]:
data['engine_unit'].unique()

array(['CC', nan], dtype=object)

- CC 한가지만 존재하므로 삭제

In [32]:
data.drop('engine_unit', axis = 1, inplace = True)

In [33]:
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248.0,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498.0,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497.0,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396.0,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298.0,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


- max_power 변수 전처리

- engine 변수와 동일한 방식으로 변환

In [34]:
data[['max_power', 'max_power_unit']] = data['max_power'].str.split(expand=True)

In [35]:
data['max_power'].head()

0        74
1    103.52
2        78
3        90
4      88.2
Name: max_power, dtype: object

- 숫자형 변수로 변환

In [38]:
data['max_power'] = data['max_power'].astype('float32')

ValueError: ignored

- bhp인 행이 있는지 확인

In [40]:
data[data['max_power'] == 'bhp']

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,max_power_unit
4933,Maruti Omni CNG,2000,80000,100000,CNG,Individual,Manual,Second Owner,10.9 km/kg,796.0,bhp,NaN,8.0,None


- Try and Excepy 구문으로 bhp인 값을 Null로 변환하는 함수 생성

In [41]:
def isFloat(value):
    try:
        num = float(value)
        return num
    except ValueError:
        return np.NaN

- 함수 적용

In [42]:
data['max_power'] = data['max_power'].apply(isFloat)

- max_power_unit 유일값 확인

In [43]:
data['max_power_unit'].unique()

array(['bhp', nan, None], dtype=object)

- bhp 이외의 단위는 없으므로 삭제

In [44]:
data.drop('max_power_unit', axis = 1, inplace=True)

In [45]:
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248.0,74.00,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498.0,103.52,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497.0,78.00,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396.0,90.00,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298.0,88.20,"11.5@ 4,500(kgm@ rpm)",5.0


- mileage 변수 전처리

- 위의 2 변수와 같은 방식 사용

In [46]:
data[['mileage', 'mileage_unit']] = data['mileage'].str.split(expand=True)

In [48]:
data['mileage'].head()

0     23.4
1    21.14
2     17.7
3     23.0
4     16.1
Name: mileage, dtype: object

In [49]:
data['mileage'] = data['mileage'].astype('float32')

- 단위가 다른 값을 동일한 컬럼에 사용하는 것은 적절하지 않으므로 추가 처리 필요
- km/kg는 킬로그램당 킬로미터, km/l은 리터당 킬로미터를 의미
- 킬로그램, 리터는 연료의 측정단위이다.
- 연료를 명시한 컬럼 fuel를 사용하여 해결해보자.
- 같은 l여도 디젤인지 휘발유인지 구별해야한다.

In [50]:
data['mileage_unit'].unique()

array(['kmpl', 'km/kg', nan], dtype=object)

- fuel 종류 확인

In [51]:
data['fuel'].unique()

array(['Diesel', 'Petrol', 'LPG', 'CNG'], dtype=object)

- 특정 시점에서 각 연료의 가격:
    + Petrol: 리터당 $80.43
    + Diesel: 리터당 $73.56
    + LPG: 킬로그램당 $40.85
    + CNG: 킬로그램당 $44.23
- mileage 변수를 각 연료별 가격으로 나누어 1달러당 주행거리 계산

### 데이터 전처리 결측치 처리와 더미 변수 변환